# Propagation Project

In [2]:
import numpy as np

## 1. GEO to LEO

$$ P(A > A_{th}) = \sum P(A > A_{th} | \theta_i) P(\theta_i) $$

csv columns:

0. Site number	
1. Satellite number	 
2. Frequency (GHz)	 
3. Elevation angle (deg)	 
4. Elevation probability ()	 
5. Time exceeded (%)	 
6. att_comb (dB)


In [4]:
aus_37_5_50 = np.genfromtxt("./data/37_austria_5_50/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
aus_37_55_90 = np.genfromtxt("./data/37_austria_55_90/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
nor_37_5_50 = np.genfromtxt("./data/37_norway_5_50/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
nor_37_55_90 = np.genfromtxt("./data/37_norway_55_90/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)

aus_37 = np.concatenate((aus_37_5_50, aus_37_55_90))
nor_37 = np.concatenate((nor_37_5_50, nor_37_55_90))

print(np.shape(aus_37))
print(np.shape(nor_37))

(432, 7)
(432, 7)
